In [ ]:
print("\n" + "="*60)
print("✅ HỦA LUYỆN VÀ LƯU MÔ HÌNH HOÀN THÀNH")
print("="*60)
print("\n📝 Các bước tiếp theo:")
print("1. ✓ Tiền xử lý dữ liệu")
print("2. ✓ Huấn luyện các mô hình")
print("3. → Đánh giá mô hình (Evaluation)")
print("4. → Phân tích Feature Importance")
print("5. → Dự đoán nguy cơ tiểu đường (Demo)")
print("="*60)

In [ ]:
import joblib
import os

# Tạo thư mục models nếu chưa tồn tại
os.makedirs('../models', exist_ok=True)

# Lưu mô hình
for model_name, model in models.items():
    model_path = f'../models/{model_name.replace(" ", "_").lower()}_model.pkl'
    joblib.dump(model, model_path)
    print(f"✓ Lưu mô hình: {model_path}")

# Lưu scaler
scaler_path = '../models/scaler.pkl'
joblib.dump(scaler, scaler_path)
print(f"✓ Lưu scaler: {scaler_path}")

# Lưu feature names
features_path = '../models/feature_names.pkl'
joblib.dump(feature_names, features_path)
print(f"✓ Lưu feature names: {features_path}")

## 4️⃣ Lưu Mô Hình và Scaler

Các mô hình sẽ được sử dụng trong phần đánh giá tiếp theo.

In [ ]:
# Vẽ biểu đồ so sánh CV scores
fig, ax = plt.subplots(figsize=(10, 6))

model_names = list(models.keys())
cv_means = [lr_cv_scores.mean(), rf_cv_scores.mean(), xgb_cv_scores.mean(), knn_cv_scores.mean()]
cv_stds = [lr_cv_scores.std(), rf_cv_scores.std(), xgb_cv_scores.std(), knn_cv_scores.std()]

colors = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12']
bars = ax.bar(model_names, cv_means, yerr=cv_stds, capsize=5, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)

ax.set_ylabel('Cross-validation F1-Score', fontweight='bold', fontsize=11)
ax.set_title('📊 So Sánh Cross-Validation F1-Scores Các Mô Hình', fontweight='bold', fontsize=13)
ax.set_ylim(0, 1)
ax.grid(True, alpha=0.3, axis='y')

# Thêm giá trị trên cột
for bar, mean in zip(bars, cv_means):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{mean:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=10)

plt.xticks(rotation=15, ha='right')
plt.tight_layout()
plt.show()

print("✓ Biểu đồ so sánh đã được vẽ")

In [ ]:
# Tóm tắt CV scores
cv_summary = {
    'Model': ['Logistic Regression', 'Random Forest', 'XGBoost', 'KNN'],
    'CV F1-Score': [
        f"{lr_cv_scores.mean():.4f}",
        f"{rf_cv_scores.mean():.4f}",
        f"{xgb_cv_scores.mean():.4f}",
        f"{knn_cv_scores.mean():.4f}"
    ],
    'Std': [
        f"±{lr_cv_scores.std():.4f}",
        f"±{rf_cv_scores.std():.4f}",
        f"±{xgb_cv_scores.std():.4f}",
        f"±{knn_cv_scores.std():.4f}"
    ]
}

cv_df = pd.DataFrame(cv_summary)
print("\n📊 Tóm tắt Cross-Validation F1-Scores:")
print(cv_df.to_string(index=False))

## 3️⃣ So Sánh Cross-Validation Scores

In [ ]:
print("\n" + "="*60)
print("✅ HỦA LUYỆN TẤT CẢ MÔ HÌNH HOÀN THÀNH")
print("="*60)
print(f"\nCác mô hình đã huấn luyện: {list(models.keys())}")
print(f"Số lượng mô hình: {len(models)}")

In [ ]:
# 4. KNN
print("\n🔹 Huấn luyện KNN...")
print("="*60)
knn_model = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
knn_model.fit(X_train_scaled, y_train)

# Cross-validation
knn_cv_scores = cross_val_score(knn_model, X_train_scaled, y_train, cv=5, scoring='f1')
print(f"  ✓ Training hoàn tất")
print(f"  ✓ Cross-validation F1-Score: {knn_cv_scores.mean():.4f} (+/- {knn_cv_scores.std():.4f})")

models['KNN'] = knn_model

In [ ]:
# 3. XGBoost
print("\n🔹 Huấn luyện XGBoost...")
print("="*60)
xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss', verbosity=0)
xgb_model.fit(X_train_scaled, y_train)

# Cross-validation
xgb_cv_scores = cross_val_score(xgb_model, X_train_scaled, y_train, cv=5, scoring='f1')
print(f"  ✓ Training hoàn tất")
print(f"  ✓ Cross-validation F1-Score: {xgb_cv_scores.mean():.4f} (+/- {xgb_cv_scores.std():.4f})")

models['XGBoost'] = xgb_model

In [ ]:
# 2. Random Forest
print("\n🔹 Huấn luyện Random Forest...")
print("="*60)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train_scaled, y_train)

# Cross-validation
rf_cv_scores = cross_val_score(rf_model, X_train_scaled, y_train, cv=5, scoring='f1')
print(f"  ✓ Training hoàn tất")
print(f"  ✓ Cross-validation F1-Score: {rf_cv_scores.mean():.4f} (+/- {rf_cv_scores.std():.4f})")

models['Random Forest'] = rf_model

In [ ]:
models = {}

# 1. Logistic Regression
print("\n🔹 Huấn luyện Logistic Regression...")
print("="*60)
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train_scaled, y_train)

# Cross-validation
lr_cv_scores = cross_val_score(lr_model, X_train_scaled, y_train, cv=5, scoring='f1')
print(f"  ✓ Training hoàn tất")
print(f"  ✓ Cross-validation F1-Score: {lr_cv_scores.mean():.4f} (+/- {lr_cv_scores.std():.4f})")

models['Logistic Regression'] = lr_model

## 2️⃣ Huấn Luyện Các Mô Hình

In [ ]:
# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n✓ Chuẩn hóa dữ liệu (StandardScaler):")
print(f"  - Mean (train): {X_train_scaled.mean(axis=0).round(2)}")
print(f"  - Std (train): {X_train_scaled.std(axis=0).round(2)}")

# Chuyển thành DataFrame cho tiện sử dụng
X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_names)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_names)

In [ ]:
# Tách X, y
X = df_clean.drop('Outcome', axis=1)
y = df_clean['Outcome']

feature_names = X.columns.tolist()

print(f"\n✓ Tách Features và Target:")
print(f"  - Features: {feature_names}")
print(f"  - Target: Outcome")

# Split train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n✓ Chia train/test (80/20):")
print(f"  - Train: {X_train.shape[0]} mẫu")
print(f"  - Test: {X_test.shape[0]} mẫu")
print(f"  - Train - Outcome=0: {(y_train==0).sum()}, Outcome=1: {(y_train==1).sum()}")
print(f"  - Test  - Outcome=0: {(y_test==0).sum()}, Outcome=1: {(y_test==1).sum()}")

In [ ]:
# Xử lý giá trị 0 không hợp lệ
print("\n📊 Xử lý giá trị 0 không hợp lệ:")
print("="*60)

df_clean = df.copy()

# Danh sách cột không nên có giá trị 0
cols_with_zero = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

for col in cols_with_zero:
    zero_count = (df_clean[col] == 0).sum()
    if zero_count > 0:
        # Tính median của những giá trị khác 0
        median_val = df_clean[df_clean[col] != 0][col].median()
        df_clean.loc[df_clean[col] == 0, col] = median_val
        print(f"  {col}: Thay {zero_count} giá trị 0 bằng median ({median_val:.2f})")

print("\n✓ Xử lý giá trị 0 hoàn tất!")

In [ ]:
# Tải dữ liệu
data_path = '../data/diabetes.csv'
df = pd.read_csv(data_path)

print(f"✓ Tải dữ liệu thành công: {df.shape[0]} mẫu, {df.shape[1]} cột")
print(f"\nTên các cột: {df.columns.tolist()}")

## 1️⃣ Tải và Tiền Xử Lý Dữ Liệu

In [ ]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

print("✓ Các thư viện đã được import thành công!")

# 🤖 Dự đoán Bệnh Tiểu Đường - Huấn Luyện Mô Hình

## Phần 2: Tiền xử lý dữ liệu & Huấn luyện các mô hình ML
- Logistic Regression
- Random Forest
- XGBoost
- KNN